# metaclass

很多中文书籍将它翻译成“元类”，事实上，meta-class 的 meta 这个词根，起源于希腊语词汇 meta，包含下面两种意思：

- “Beyond”，例如技术词汇 metadata，意思是描述数据的超越数据；
- “Change”，例如技术词汇 metamorphosis，意思是改变的形态。

咱就从它“超越类”和“变形类”的“超越变形”特性着手吧

## YAML

YAML 的动态 序列化 / 反序列化 是由 metaclass 实现的（极少人知道，可能只有 1%）


In [1]:
import yaml

class Monster(yaml.YAMLObject):
    # 如果没有显示赋值 yaml_tag，
    # 那么就是 !!python/object:__main__.Monster
    yaml_tag = u'!Monster'

    def __init__(self, name, hp, ac, attacks):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.attacks = attacks

    def __repr__(self):
        # return f'{self.__class__.__name__}(name={self.name}, hp={self.hp}, ac={self.ac}, attacks={self.attacks})'
        return "%s(name=%r, hp=%r, ac=%r, attacks=%r)" % (
            self.__class__.__name__, self.name, self.hp, self.ac,
            self.attacks)


data = yaml.load("""
--- !Monster
name: Cave spider
hp: [2,6]    # 2d6
ac: 16
attacks: [BITE, HURT]
""")
print(yaml.dump(data))
print(data.hp)

# print(Monster(name='Cave lizard', hp=[3,6], ac=16, attacks=['BITE','HURT']))
print(yaml.dump(Monster(name='Cave lizard', hp=[3,6], ac=16, attacks=['BITE','HURT'])))

!Monster
ac: 16
attacks:
- BITE
- HURT
hp:
- 2
- 6
name: Cave spider

[2, 6]
!Monster
ac: 16
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: Cave lizard



上面这个例子，为了实现 序列化 和 反序列化，需要在 Monster 类定义后面加上下面这行代码：`add_constructor(Monster)`。

对于 YAML 的使用者来说，每一个 YAML 的可逆序列化的类 Foo 定义后，都需要加上一句话，`add_constructor(Foo)`。这无疑给开发者增加了麻烦，也更容易出错，毕竟开发者很容易忘了这一点。那么，更优的实现方式是什么样呢？

翻阅 YAML 的源码

```yaml
# Python 2/3 相同部分
class YAMLObjectMetaclass(type):
    def __init__(cls, name, bases, kwds):
        super(YAMLObjectMetaclass, cls).__init__(name, bases, kwds)
        if 'yaml_tag' in kwds and kwds['yaml_tag'] is not None:
            cls.yaml_loader.add_constructor(cls.yaml_tag, cls.from_yaml)
    # 省略其余定义

# Python 3
class YAMLObject(metaclass=YAMLObjectMetaclass):
    yaml_loader = Loader
    # 省略其余定义

# Python 2
class YAMLObject(object):
    __metaclass__ = YAMLObjectMetaclass
    yaml_loader = Loader
    # 省略其余定义
```

可以看出无论是 python 2 还是python 3， YAMLObject 把 metaclass 都声明成了 YAMLObjectMetaclass。

这行代码 `cls.yaml_loader.add_constructor(cls.yaml_tag, cls.from_yaml)` 是魔法发生的地方。

YAML 应用 metaclass，拦截了所有 YAMLObject 子类的定义，把代码 `add_constructor(Foo)` 给自动加上了。


## Python 底层语言设计层面是如何实现 metaclass 的？

### 第一，所有的 Python 的用户定义类，都是 type 这个类的实例。

下面这个例子：

instance 是 MyClass 的实例，而 MyClass 不过是“上帝”type 的实例。

In [2]:
class MyClass:
  pass

instance = MyClass()

print('instance: ', type(instance))
print('MyClass: ', type(MyClass))


instance:  <class '__main__.MyClass'>
MyClass:  <class 'type'>


### 第二，用户自定义类，只不过是 type 类的 `__call__` 运算符重载。

简单来说，当我们定义

```python
class MyClass:
  data = 1
```

真正执行的是

```python
class = type(classname, superclasses, attributedict)
```

等号右边的 `type(classname, superclasses, attributedict)`，就是 type 的 `__call__` 运算符重载，它会进一步调用：

```python
type.__new__(typeclass, classname, superclasses, attributedict)
type.__init__(class, classname, superclasses, attributedict)
```


In [3]:
class MyClass:
  data = 1
  
instance = MyClass()
MyClass, instance, instance.data

(__main__.MyClass, <__main__.MyClass at 0x7f84b0030210>, 1)

In [4]:
# type(classname, superclasses, attributedict)
MyClass = type('MyClass', (), {'data': 1})

instance = MyClass()
MyClass, instance, instance.data

(__main__.MyClass, <__main__.MyClass at 0x7f84f835d110>, 1)

### 第三，metaclass 是 type 的子类，通过替换 type 的 `__call__` 运算符重载机制，“超越变形”正常的类。

理解了以上几点，我们就会明白，正是 Python 的类创建机制，给了 metaclass 大展身手的机会。一旦你把一个类型 MyClass 的 metaclass 设置成 MyMeta，MyClass 就不再由原生的 type 创建，而是会调用 MyMeta 的 `__call__` 运算符重载。

```python
class = type(classname, superclasses, attributedict) 
# 变为了
class = MyMeta(classname, superclasses, attributedict)
```

现在可以理解上面 YAML 的例子中，利用 YAMLObjectMetaclass 的 `__init__` 方法，为所有 YAMLObject 子类偷偷执行 `add_constructor()`。

## 使用 metaclass 的风险

凡事有利必有弊，尤其是 metaclass 这样“逆天”的存在。正如你所看到的那样，metaclass 会"扭曲变形"正常的 Python 类型模型。所以，如果使用不慎，对于整个代码库造成的风险是不可估量的。


## 思考题

Python 装饰器 和 metaclass，它们都能干预正常的 Python 类型机制。那么，你觉得装饰器和 metaclass 有什么区别呢？

metaclass 通过重载 `__call__` 来修改类的行为，而装饰器是通过将原函数进行包装来修改原函数的功能。

两者都对原有的功能进行了扩展，但是实现思路不一样，metaclass 是从类型机制这个角度，而装饰器主要从函数的角度，装饰器更加轻量级（因此更加适合业务应用层面），而 metaclass 更加重量级一些（可以看成是“大招”，也因此更加适合框架层面）。


## 附加示例


In [7]:
from my_meta import run_test

run_test()

==>MyMeta.__new__
   cls.__name__:  MyMeta
==>MyMeta.__init__
   cls.__name__:  Foo
   attributedict:  {'__module__': 'my_meta', '__qualname__': 'Foo', 'yaml_tag': '!Foo', '__init__': <function Foo.__init__ at 0x7f84f889e950>, '__new__': <function Foo.__new__ at 0x7f84f889e9e0>, '__repr__': <function Foo.__repr__ at 0x7f84f889ea70>}
==>MyMeta.__call__
Foo.__new__
Foo.__init__
!!python/object:my_meta.Foo
greeting: hello world
name: foo

----------
Foo2.__new__
==>MyClass.__init__
   self.__name__:  Foo2
Foo2.__init__
foo2
!Foo2
name: foo2

----------
==>MyClass.__new__
   cls.__name__:  MyClass
==>MyClass.__init__
   self.__name__:  MyClass
super_foo3
!MyClass
name: super_foo3

----------
Foo.__new__
Foo.__repr__
Foo(name=Cave spider, greeting=Good Afternoon)
!!python/object:__main__.Foo
greeting: Good Afternoon
name: Cave spider



In [9]:
import os
from yaml_test import get_config

# file_path = os.path.join(os.getcwd(), 'yaml_test.py')
# %load file_path

config_path = os.path.join(os.getcwd(), 'test_config.yaml')

config = get_config(config_file=config_path)
config

{'data_root': '/home/fengyifan/data/features',
 'modelnet40_ft': '/home/fengyifan/data/features/ModelNet40_mvcnn_gvcnn.mat',
 'ntu2012_ft': '/home/fengyifan/data/features/NTU2012_mvcnn_gvcnn.mat',
 'graph_type': 'hypergraph',
 'K_neigs': [10],
 'm_prob': 1.0,
 'is_probH': True,
 'use_mvcnn_feature_for_structure': True,
 'use_gvcnn_feature_for_structure': True,
 'on_dataset': 'ModelNet40',
 'use_mvcnn_feature': False,
 'use_gvcnn_feature': True,
 'result_root': '/home/fengyifan/result/hgnn',
 'result_sub_folder': '/home/fengyifan/result/hgnn/hypergraph_ModelNet40',
 'ckpt_folder': '/home/fengyifan/result/hgnn/ckpt',
 'max_epoch': 600,
 'n_hid': 128,
 'lr': 0.001,
 'milestones': [100],
 'gamma': 0.9,
 'drop_out': 0.5,
 'print_freq': 50,
 'weight_decay': 0.0005,
 'decay_step': 200,
 'decay_rate': 0.7}